<a href="https://colab.research.google.com/github/tadeugomes/mba_enap_python/blob/main/CD/Atividades_Exercicios/Aula3_Exercicio3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício 3.1

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')


Authenticated


In [ ]:
import pandas as pd
## Defina o id do seu projeto no bigquery!!!!!
project_id = 'enap-mba-470912 ' # Defina o id do seu projeto no bigquery!!!!!
## Defina o id do seu projeto no bigquery!!!!!

df = pd.io.gbq.read_gbq('''
SELECT ... ''', project_id=project_id)

df.head()

,ano,sigla_uf,id_municipio,populacao,nome_municipio,pib,pibpercapita
0,2002,RO,1100015,27237.0,Alta Floresta D'oeste,111290995,4086.022506
1,2003,RO,1100015,27563.0,Alta Floresta D'oeste,143222381,5196.182600
2,2004,RO,1100015,29001.0,Alta Floresta D'oeste,173990790,5999.475535
3,2005,RO,1100015,28629.0,Alta Floresta D'oeste,167127355,5837.694471
4,2006,RO,1100015,29005.0,Alta Floresta D'oeste,168805060,5819.860714


### Minha Query

In [3]:
import pandas as pd
## Defina o id do seu projeto no bigquery!!!!!
project_id = 'enap-mba-470912' # Defina o id do seu projeto no bigquery!!!!!
## Defina o id do seu projeto no bigquery!!!!!

df = pd.io.gbq.read_gbq('''
SELECT
  pib.ano AS ano,
  uf.sigla AS sigla_uf,
  mun.id_municipio,
  pop.populacao,
  mun.nome AS nome_municipio,
  pib.pib,
  ROUND(CAST(pib.pib AS FLOAT64)/NULLIF(CAST(pop.populacao AS FLOAT64),0), 6) AS pibpercapita
FROM
  `basedosdados.br_ibge_pib.municipio` AS pib
JOIN
  `basedosdados.br_ibge_populacao.municipio` AS pop
    ON pib.id_municipio = pop.id_municipio AND pib.ano = pop.ano
JOIN
  `basedosdados.br_bd_diretorios_brasil.municipio` AS mun
    ON pib.id_municipio = mun.id_municipio
JOIN
  `basedosdados.br_bd_diretorios_brasil.uf` AS uf
    ON mun.id_uf = uf.id_uf
WHERE
  pib.ano BETWEEN 2002 AND 2018
ORDER BY
  ano, sigla_uf, nome_municipio
''', project_id=project_id)

df.head()

/tmp/ipython-input-679519711.py:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.io.gbq.read_gbq('''


,ano,sigla_uf,id_municipio,populacao,nome_municipio,pib,pibpercapita
0,2002,AC,1200013,8454,Acrelândia,39622000,4686.775491
1,2002,AC,1200054,3611,Assis Brasil,12789000,3541.678205
2,2002,AC,1200104,17649,Brasiléia,72997000,4136.041702
3,2002,AC,1200138,6382,Bujari,26743000,4190.379191
4,2002,AC,1200179,5814,Capixaba,32492000,5588.579291


In [ ]:
## Explique o que fez na sua query e por que.
# Fiz o left join da população porque essa tabela tem mais anos.
# Tive que fazer o distinct da tabela geobr_mapas porque essa tabela granularidade de setor censitário.

### Explicar o que fiz na query

### Explicar o que fiz na query

Esta query SQL busca dados de diferentes tabelas públicas do BigQuery (`basedosdados`) para calcular o Produto Interno Bruto (PIB) per capita para os municípios brasileiros entre os anos de 2002 e 2018.

Aqui está uma quebra do que a query faz:

1.  **`SELECT ...`**: Especifica as colunas que serão retornadas no resultado final.
    *   `pib.ano AS ano`: Seleciona o ano da tabela de PIB e o renomeia como `ano`.
    *   `uf.sigla AS sigla_uf`: Seleciona a sigla da UF da tabela de UFs e a renomeia como `sigla_uf`.
    *   `mun.id_municipio`: Seleciona o ID do município da tabela de municípios.
    *   `pop.populacao`: Seleciona a população da tabela de população.
    *   `mun.nome AS nome_municipio`: Seleciona o nome do município da tabela de municípios e o renomeia como `nome_municipio`.
    *   `pib.pib`: Seleciona o valor do PIB da tabela de PIB.
    *   `ROUND(CAST(pib.pib AS FLOAT64)/NULLIF(CAST(pop.populacao AS FLOAT64),0), 6) AS pibpercapita`: Calcula o PIB per capita.
        *   `CAST(... AS FLOAT64)`: Converte os valores de PIB e população para números decimais para garantir uma divisão precisa.
        *   `NULLIF(CAST(pop.populacao AS FLOAT64),0)`: Garante que a divisão não ocorra por zero, retornando NULL se a população for zero.
        *   `ROUND(..., 6)`: Arredonda o resultado do PIB per capita para 6 casas decimais.
        *   `AS pibpercapita`: Nomeia a coluna resultante como `pibpercapita`.

2.  **`FROM basedosdados.br_ibge_pib.municipio AS pib`**: Define a tabela `basedosdados.br_ibge_pib.municipio` como a tabela principal e a renomeia como `pib` para facilitar a referência.

3.  **`JOIN ... ON ...`**: Conecta a tabela principal (`pib`) a outras tabelas usando colunas comuns:
    *   `JOIN basedosdados.br_ibge_populacao.municipio AS pop ON pib.id_municipio = pop.id_municipio AND pib.ano = pop.ano`: Junta com a tabela de população (`pop`) onde o ID do município e o ano coincidem.
    *   `JOIN basedosdados.br_bd_diretorios_brasil.municipio AS mun ON pib.id_municipio = mun.id_municipio`: Junta com a tabela de diretórios de municípios (`mun`) onde o ID do município coincide.
    *   `JOIN basedosdados.br_bd_diretorios_brasil.uf AS uf ON mun.id_uf = uf.id_uf`: Junta com a tabela de diretórios de UFs (`uf`) onde o ID da UF coincide (conectando através da tabela de municípios `mun`).

4.  **`WHERE pib.ano BETWEEN 2002 AND 2018`**: Filtra os resultados para incluir apenas os registros onde o ano está entre 2002 e 2018, inclusive.

5.  **`ORDER BY ano, sigla_uf, nome_municipio`**: Ordena os resultados primeiro pelo ano, depois pela sigla da UF e, por fim, pelo nome do município em ordem alfabética.

Em resumo, a query combina dados de PIB, população, municípios e UFs para calcular o PIB per capita para cada município em cada ano entre 2002 e 2018.

## Submeta o seu dataframe criando uma tabela no BigQuery

In [4]:
df.to_gbq("enapdatasets.pibpercapita",
              project_id=project_id,
              chunksize=40000,
              if_exists='replace',
              )

/tmp/ipython-input-3064127241.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq("enapdatasets.pibpercapita",
100%|██████████| 1/1 [00:00<00:00, 10356.31it/s]


#### O comando acima cria um dataset chamado enapdatasets e dentro dele uma tabela chamada pibpercapita.
#### Por padrão, os dados são guardados fisicamente nos EUA, onde também estão guardados os dados do projeto basedosdados.
#### Não é possível fazer join de dados guardados em locais distintos

##### Aqui há uma demonstração de como [criar um dataset manualmente](https://youtu.be/h_HRXdket7U)